In [ ]:
import json
from decimal import Decimal
import boto3

PRODUCT_FILE="dataset_amazon-reviews-apple-watch_modified_sample.csv"

# the below function reads in the data and does the required analysis and uploads 
# it to a dynamodb table

def lambda_handler(event, context):
    s3=boto3.client("s3")
    bucket="bigdatatestrevsamp"
    key=PRODUCT_FILE
    file=s3.get_object(Bucket=bucket,Key=key)
    reviews=str(file['Body'].read())
    
    reviews_list=reviews.split("\\r\\n")
    
    comprehend = boto3.client("comprehend")
    n=1
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('trends_sentiment_phrases')
    
    for i in reviews_list :
        response1 = comprehend.detect_sentiment(Text=i, LanguageCode="en")
        response2 = comprehend.detect_key_phrases(Text=i, LanguageCode="en")
        n=n+1
        key_phrases=[]
        
        dynamodb = boto3.resource('dynamodb')
        table = dynamodb.Table('sentiment_analysis')
        s_item = json.loads(json.dumps(response1['SentimentScore']), parse_float=Decimal)
        table.put_item(
            Item={
                'review_num': n,
                'Review': i,  
                'All_Sentiment':str(response1['SentimentScore']),
                'Sentiment_Confidence':s_item[response1['Sentiment'].capitalize()],
                'Overall_Sentiment':response1['Sentiment'],
                'Key_Phrases':str(key_phrases)
            })
        
    return "Done"
    
    
